In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
# !unzip drive/MyDrive/sasw_data/data1.zip
# !unzip drive/MyDrive/sasw_data/data2.zip
!unzip drive/MyDrive/sasw_data/data3.zip

Archive:  drive/MyDrive/sasw_data/data3.zip
  inflating: fake_or_real_news.csv   


In [9]:
from DataLoader import DataLoader

ROOT = "data/"
datasets = ["data1", "data2", "data3"]

dataset = DataLoader(ROOT, datasets)
dataset.dataset

Loading datasets...
Loading from: data/data1/train.csv
Loaded: 20800 rows

Loading from: data/data2/Fake.csv
Loaded: 44281 rows

Loading from: data/data2/True.csv
Loaded: 65698 rows

Loading from: data/data3/fake_or_real_news.csv
Loaded: 72033 rows

Total loaded 5 rows

Cleaning dataset...


  0%|          | 0/72033 [00:00<?, ?it/s]/content/DataLoader.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataset["clean_words"][i] = self.Preprocess("{} {}".format(self.dataset["title"][i], self.dataset["text"][i]))
/content/DataLoader.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataset["clean"][i] = " ".join(self.dataset["clean_words"][i])
100%|██████████| 72033/72033 [03:46<00:00, 317.91it/s]


DATASET: train 0.6 test 0.2 val 0.2


,title,text,label,clean_words,clean
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[house, aide, comey, letter, jason, chaffetz, ...",house aide comey letter jason chaffetz tweeted...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,"[flynn, hillary, clinton, woman, campus, breit...",flynn hillary clinton woman campus breitbart f...
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,"[truth, fired, truth, fired, october, tension,...",truth fired truth fired october tension intell...
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,"[civilians, killed, single, airstrike, identif...",civilians killed single airstrike identified v...
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,"[iranian, woman, jailed, fictional, unpublishe...",iranian woman jailed fictional unpublished sto...
...,...,...,...,...,...
72028,State Department says it can't find emails fro...,The State Department told the Republican Natio...,1,"[state, department, says, emails, clinton, spe...",state department says emails clinton specialis...
72029,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,0,"[stand, plutocratic, pentagon, stand, plutocra...",stand plutocratic pentagon stand plutocratic p...
72030,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,0,"[anti, trump, protesters, tools, oligarchy, in...",anti trump protesters tools oligarchy informat...
72031,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",1,"[ethiopia, obama, seeks, progress, peace, secu...",ethiopia obama seeks progress peace security e...


In [10]:
for i,doc in enumerate(dataset.train_sequences[:2]):
     print("The padded encoding for document",i+1," is : ",doc[:100])

The padded encoding for document 1  is :  [ 5105 12218  1083   237   172  5267  3741   366     2   232   227  1013
   237   172   264    21  1083   171  5267  3741   366     2   232   227
  1013  3569  4307   302  5105 12218   488  1285 13479  2759  1641  1497
     2 12218   242   276   478   475 12369  1846  3992  2759   441  1652
     2    68   677  8861  2777   607 13006   987    18 17616   553  3917
    56  2809    29   849   607 10136 35152     3     2   232   227   686
   381   227  1654   227  9934   914   232     3    25  2552    29   849
  1800 12218   167     7   404   259  1247   277  9793   772  1247   232
   227 15736  2325    23]
The padded encoding for document 2  is :  [  426  3254 13381   751   295  2170  3775   210    33    31   426  3190
   176  3971    76    65 10257    26  2170   257  3775   210    11   721
   559   382    76   171   283   109   176    52  2170  3808  1162  1035
   288    32  1162    49   458    98   642  2170   257   963  1266   554
   133   261  

In [11]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

# Sequential Model
model = Sequential()
# embeddidng layer
model.add(Embedding(len(dataset.list_of_words), output_dim = 128))
# model.add(Embedding(total_words, output_dim = 240))
# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(128)))

# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         28517120  
                                                                 
 bidirectional (Bidirectiona  (None, 256)              263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 28,813,313
Trainable params: 28,813,313
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_train = np.asarray(dataset.y_train)




model.fit(dataset.train_sequences, y_train, batch_size = 64, validation_split = 0.2, epochs = 1)

Epoch 1/5
127/577 [=====>........................] - ETA: 2:50:46 - loss: 0.4112 - acc: 0.8055